In [1]:
from dataset_management import DatasetManager, SimpleEncoder, EncodingType
from typing import List

In [2]:
import chess
import chess.pgn
import sys

In [3]:
import tqdm
import random
import itertools
import os

In [4]:
def read_games(path : str) -> List[chess.pgn.Game]:
    with open(path, 'r') as file:
        print(f"Processing file {path}")
        game = chess.pgn.read_game(file)
        games = []
        nr = 0
        while game != None:
            if nr % 100 == 0:
                print(f"{nr} games processed so far...", end='\r')
            games.append(game)
            game = chess.pgn.read_game(file)
            nr += 1
        print(100*' ', end='\r')
        print("Processing finished")
        print(f"{nr} games have been loaded from the file {path}")
    return games

In [5]:
def game_equality(g1 : chess.pgn.Game, g2 : chess.pgn.Game, s : str):
    for i, (m1, m2) in enumerate(zip(g1.mainline_moves(), g2.mainline_moves())):
        assert m1.uci() == m2.uci(), \
        s + f'move number: {i}, first game: {m1.uci()}, second game: {m2.uci()}'

def test_encoder(games : List[chess.pgn.Game], encoder : EncodingType):
    for j, game in tqdm.tqdm(enumerate(games)):
        encoding = encoder.encode_pgn(game)
        decoded = encoder.decode_to_pgn(encoding)
        game_equality(game, decoded, f'game number: {j}')

In [6]:
def test_game_generator_correctness(games : List[chess.pgn.Game], dataset : DatasetManager, n : int):
    print(f'Running {n} tests')
    total_games_tested = 0
    for t in tqdm.tqdm(range(n)):
        l, r = random.randint(0, len(games)-1), random.randint(0, len(games)-1)
        if r < l:
            l, r = r, l
        for i, game in enumerate(dataset.games(l, r)):
            original_game = games[l+i]
            game_equality(original_game, game, f'interval: [{l}, {r}], game number: {l+i}')
        total_games_tested += r-l+1
        tqdm.tqdm.write(f'{total_games_tested} games checked so far')
    print(f'Finished, {n} tests passed, {total_games_tested} games checked in total')
            

In [7]:
def test_position_selector_correctness(games : List[chess.pgn.Game], dataset : DatasetManager, n : int):
    print(f'Running {n} tests')
    total_moves_on_prefix = list(itertools.accumulate(map(lambda x: len(list(x.mainline_moves())), games), initial=0))
    total_moves_tested = 0
    for t in tqdm.tqdm(range(n)):
        game_id = random.randint(0, len(games)-1)
        for i, m in enumerate(games[game_id].mainline_moves()):
            position_id = total_moves_on_prefix[game_id] + i
            move_id, g = dataset.game_by_position_id(position_id)
            assert i == move_id, "wrong move index"
            game_equality(games[game_id], g, f'position id: {position_id}')
            total_moves_tested += 1
        if t % 10 == 0:
            tqdm.tqdm.write(f'{total_moves_tested} positions checked so far')
    print(f'Finished, {n} tests passed, {total_moves_tested} positions checked in total')


In [8]:
dataset = DatasetManager('my_dataset', create_empty=True)

In [9]:
encoder = SimpleEncoder()

In [10]:
dataset.add_pgn_file(os.path.join('Lichess Elite Database', 'lichess_elite_2016-01.pgn'))

max_number_of_games: 1000
Chunk len: 1000
number_of_positions: 78882
max_number_of_games: 2000
Chunk len: 1000
number_of_positions: 158447
max_number_of_games: 3000
Chunk len: 1000
number_of_positions: 238136
max_number_of_games: 4000
Chunk len: 1000
number_of_positions: 320212
max_number_of_games: 5000
Chunk len: 1000
number_of_positions: 400211
max_number_of_games: 6000
Chunk len: 1000
number_of_positions: 481340
max_number_of_games: 7000
Chunk len: 1000
number_of_positions: 560873
max_number_of_games: 8000
Chunk len: 1000
number_of_positions: 639921
max_number_of_games: 9000
Chunk len: 1000
number_of_positions: 718378
max_number_of_games: 10000
Chunk len: 91
number_of_positions: 725493


In [11]:
games = read_games(os.path.join('Lichess Elite Database', 'lichess_elite_2016-01.pgn'))

Processing file /home/ereonzis/Dokumenty/ProjectPhoenix/dataset/Lichess Elite Database/lichess_elite_2016-01.pgn
Processing finished                                                                                 
9091 games have been loaded from the file /home/ereonzis/Dokumenty/ProjectPhoenix/dataset/Lichess Elite Database/lichess_elite_2016-01.pgn


In [12]:
test_encoder(games, encoder)

9091it [00:16, 541.26it/s]


In [13]:
test_position_selector_correctness(games, dataset, 1000)

Running 1000 tests


  0%|          | 2/1000 [00:00<03:11,  5.21it/s]

88 positions checked so far


  1%|          | 11/1000 [00:02<04:05,  4.03it/s]

940 positions checked so far


...

76680 positions checked so far


 99%|█████████▉| 991/1000 [03:48<00:01,  4.77it/s]

77279 positions checked so far


100%|██████████| 1000/1000 [03:52<00:00,  4.31it/s]

Finished, 1000 tests passed, 78175 positions checked in total


In [14]:
test_game_generator_correctness(games, dataset, 10)

Running 10 tests


 10%|█         | 1/10 [00:05<00:45,  5.04s/it]

3015 games checked so far


 10%|█         | 1/10 [00:07<01:09,  7.69s/it]


KeyboardInterrupt: 

In [ ]:
#Testing loading existings dataset
dataset = DatasetManager('my_dataset', create_empty=False)

In [ ]:
test_game_generator_correctness(games, dataset, 10)

In [ ]:
dataset.add_pgn_file(os.path.join('Lichess Elite Database', 'lichess_elite_2015-11.pgn'))

In [ ]:
games.extend(read_games(os.path.join('Lichess Elite Database', 'lichess_elite_2015-11.pgn')))

In [ ]:
test_game_generator_correctness(games, dataset, 10)